# Arbol de Decisión Para Clasificación

1. Índice Gini  


Ejemplo de conjunto de datos de detecciónd e billetes falsos.  

$\sum_{c=1}^C \hat{\pi}_c(1-\hat{\pi}_c) = 1 - \sum_{c}\hat{\pi}^2_c$

donde $\pi$ es la proporción 

$\hat{\pi}_c = \frac{1}{|D|}\sum_{i\in D} I (y_i=c)$

Una función de costo utilizada para evaluar la pureza de las divisiones en el conjunto de datos es el **índice Gini**

$\sum_{c=1}^C \hat{\pi}_c(1-\hat{\pi}_c) = 1 - \sum_{c}\hat{\pi}^2_c$

El índice Gini es un indicador de pureza que describe que tan bien las clases están separadas dado un parámetro y valor de umbral de separación.


In [205]:
def gini_index(groups, classes):
    
    # contar todas las muestras contenidas de las dos particiones dado un punto de corte ("split point")
    n_instances = float(sum([len(group) for group in groups]))

    # acumular gini indices ponderados por cada grupo
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # evitar division entre cero
        if size == 0:
            continue
        score = 0.0

        # suma del score de cada clase, aquí es donde esta la posibilidad de dividir entre cero
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p

        # ponderar el score del grupo por su tamaño relativo
        ponderacion = (size / n_instances)
        gini += (1.0 - score) * ponderacion
    return gini

## 2. División de de la base de datos dado una dimensión $j\in {1,\dots,D}$ y un valor de umbral 

In [206]:
# dividir la base de datos basado en el atributo apuntado por el indice y el valor del atributo que es utilizado como umbral (indice, umbral)
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

Seleccionar el la mejor dimensión y valor de umbral de separación

In [207]:
# Seleccionar el mejor punto de corte (indice, umbral)
def get_split(dataset):
    
    #coleccionar todas las etiquetas posibles (0,1)
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    
    #por cada indice
    dimensiones = len(dataset[0])-1
    for index in range(dimensiones):
        for row in dataset:
            
            #probar por cada indice, umbral
            umbral = row[index]
            groups = test_split(index, umbral , dataset)

            gini = gini_index(groups, class_values)
            print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            
            #guardamos el mejor punto de corte definido por (indice, umbral)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}
 

## 3. Ejemplo

In [208]:
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]

split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

X1 < 2.771 Gini=0.444
X1 < 1.729 Gini=0.500
X1 < 3.678 Gini=0.286
X1 < 3.961 Gini=0.167
X1 < 2.999 Gini=0.375
X1 < 7.498 Gini=0.286
X1 < 9.002 Gini=0.375
X1 < 7.445 Gini=0.167
X1 < 10.125 Gini=0.444
X1 < 6.642 Gini=0.000
X2 < 1.785 Gini=0.500
X2 < 1.170 Gini=0.444
X2 < 2.813 Gini=0.320
X2 < 2.620 Gini=0.417
X2 < 2.209 Gini=0.476
X2 < 3.163 Gini=0.167
X2 < 3.339 Gini=0.444
X2 < 0.477 Gini=0.500
X2 < 3.235 Gini=0.286
X2 < 3.320 Gini=0.375
Split: [X1 < 6.642]


## 4. Implementación de la construcción de un árbol de decisión para clasificación

In [209]:
# Crear un nodo terminal con un valor
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return min(set(outcomes), key=outcomes.count)


La construcción de un árbol de decisión involucra la llamada de la función get_split de manera recursiva en los grupos creados para cada nodo

In [210]:

# Split Crea divisiones para un nodo o genera un nodo terminal.
def split(node, max_depth, min_size, depth):
    
    # node['groups'] contiene los grupos generados por get_split
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
# process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [211]:
# función que construye un árbol de decisión
def build_tree(dataset, max_depth, min_size):
    root = get_split(dataset)
    split(root, max_depth, min_size, 1)
    return root

In [212]:
# Imprimir el árbol de decisión modo texto
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [213]:
dataset = [[2.771244718,1.784783929,0],
[1.728571309,1.169761413,0],
[3.678319846,2.81281357,0],
[3.961043357,2.61995032,0],
[2.999208922,2.209014212,0],
[7.497545867,3.162953546,1],
[9.00220326,3.339047188,1],
[7.444542326,0.476683375,1],
[10.12493903,3.234550982,1],
[6.642287351,3.319983761,1]]



In [214]:
tree = build_tree(dataset, 4, 1)

X1 < 2.771 Gini=0.444
X1 < 1.729 Gini=0.500
X1 < 3.678 Gini=0.286
X1 < 3.961 Gini=0.167
X1 < 2.999 Gini=0.375
X1 < 7.498 Gini=0.286
X1 < 9.002 Gini=0.375
X1 < 7.445 Gini=0.167
X1 < 10.125 Gini=0.444
X1 < 6.642 Gini=0.000
X2 < 1.785 Gini=0.500
X2 < 1.170 Gini=0.444
X2 < 2.813 Gini=0.320
X2 < 2.620 Gini=0.417
X2 < 2.209 Gini=0.476
X2 < 3.163 Gini=0.167
X2 < 3.339 Gini=0.444
X2 < 0.477 Gini=0.500
X2 < 3.235 Gini=0.286
X2 < 3.320 Gini=0.375
X1 < 2.771 Gini=0.000
X1 < 1.729 Gini=0.000
X1 < 3.678 Gini=0.000
X1 < 3.961 Gini=0.000
X1 < 2.999 Gini=0.000
X2 < 1.785 Gini=0.000
X2 < 1.170 Gini=0.000
X2 < 2.813 Gini=0.000
X2 < 2.620 Gini=0.000
X2 < 2.209 Gini=0.000
X1 < 2.771 Gini=0.000
X1 < 3.678 Gini=0.000
X1 < 3.961 Gini=0.000
X1 < 2.999 Gini=0.000
X2 < 1.785 Gini=0.000
X2 < 2.813 Gini=0.000
X2 < 2.620 Gini=0.000
X2 < 2.209 Gini=0.000
X1 < 7.498 Gini=0.000
X1 < 9.002 Gini=0.000
X1 < 7.445 Gini=0.000
X1 < 10.125 Gini=0.000
X1 < 6.642 Gini=0.000
X2 < 3.163 Gini=0.000
X2 < 3.339 Gini=0.000
X2 < 0.4

In [215]:
print_tree(tree)

[X1 < 6.642]
 [X1 < 2.771]
  [0]
  [X1 < 2.771]
   [0]
   [0]
 [X1 < 7.498]
  [X1 < 7.445]
   [1]
   [1]
  [X1 < 7.498]
   [1]
   [1]


In [189]:
# Predicción con el árbol de decisión.
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

Referencias:

https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/